In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import os; import re

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
from sklearn.cross_validation import KFold

In [6]:
from sklearn.grid_search import GridSearchCV

In [7]:
import time
import datetime

In [8]:
from sklearn.linear_model import LogisticRegressionCV

In [9]:
from sklearn.preprocessing import StandardScaler

In [30]:
from sklearn.decomposition import PCA

In [10]:
features = pd.read_csv('./_features/features.csv', index_col='match_id')

In [11]:
dict_names = os.listdir("./_data/data/dictionaries")

###### create dict of dicts with object description

In [12]:
dicts = dict()
for i,d in enumerate(dict_names):
    dicts[re.sub(r'(.*)\.csv',r'\1',d)] = pd.read_csv('./_data/data/dictionaries/%s' % d)
#print dicts

###### drop final attributes

In [13]:
X = features.drop(features.columns[-6:],1)

###### found skips

In [14]:
counts=X.count()

In [15]:
skips=counts[counts.values < 97230].keys()

###### не в каждом матче FB происходит в первые 5 минут, не в каждом матче летучий курьер приобретается одной из команд в первые 5 минут

In [16]:
skips[[0,-2]]

Index([u'first_blood_time', u'dire_flying_courier_time'], dtype='object')

###### replace na items to ZERO

In [17]:
X_na = X.fillna(0)

###### target var - radiant_win

In [18]:
y=features.radiant_win

###### create cross validator

In [19]:
cv = KFold(y.size,shuffle=True, n_folds=5)

In [20]:
sc=StandardScaler()

Xs=sc.fit_transform(X_na)

In [21]:
keys=list(X.keys()[pd.Series(X.keys()).str.contains('hero', na=False)])

keys.append('lobby_type')

Xdr = X_na.drop(keys,1)

In [22]:
Xdr2=Xdr.drop(X.keys()[pd.Series(X.keys()).str.contains('first_blood_player', na=False)],1)

In [23]:
scdr=StandardScaler()

Xsdr=scdr.fit_transform(Xdr2)

In [48]:
est1 = LogisticRegressionCV(Cs=20,cv=cv,scoring='roc_auc',n_jobs=-1)

In [49]:
start_time = datetime.datetime.now()
est1.fit(Xsdr,y)
print 'Time elapsed:', datetime.datetime.now() - start_time    

Time elapsed: 0:00:17.918000


In [50]:
est1.scores_[1].max()

0.72075742226010331

In [38]:
est1.scores_[1].max()

0.72075651602744528

In [24]:
ky=list(X.keys()[pd.Series(X.keys()).str.contains('hero', na=False)])

frames = []
for i in ky:
    frames.append(pd.DataFrame(X[i]).rename(index=str, columns={i: "hero"}))

all_hero = pd.concat(frames,ignore_index=True)
N=all_hero['hero'].unique().size

In [25]:
N=112

In [26]:
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in xrange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [27]:
XX=np.column_stack((Xsdr,X_pick))

In [226]:
list(range(10,21))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [28]:
estD = LogisticRegressionCV(Cs=15,cv=cv,scoring='roc_auc',n_jobs=-1)

In [29]:
start_time = datetime.datetime.now()
estD.fit(XX,y)
print 'Time elapsed:', datetime.datetime.now() - start_time    

Time elapsed: 0:00:54.073000


In [30]:
estD.scores_[1].max()

0.754412342988241

In [177]:
estD.scores_[1].max()

0.75663728773447603

In [171]:
est.get_params

<bound method LogisticRegressionCV.get_params of LogisticRegressionCV(Cs=15, class_weight=None,
           cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='ovr', n_jobs=-1, penalty='l2',
           random_state=None, refit=True, scoring='roc_auc',
           solver='lbfgs', tol=0.0001, verbose=0)>

In [28]:
est = LogisticRegressionCV(Cs=15,cv=cv,scoring='roc_auc',n_jobs=-1)

In [29]:
start_time = datetime.datetime.now()
est.fit(Xs,y)
print 'Time elapsed:', datetime.datetime.now() - start_time    

Time elapsed: 0:00:19.256000


In [33]:
est.scores_[1].max()

0.7206460881333201

In [28]:
X_pick.shape

(97230L, 112L)

In [95]:
pest = PCA(n_components=180)
pest.fit(XX.T)
#est.components_
disp=pest.explained_variance_ratio_
disp.sort()
rd=disp[::-1]
rd.cumsum()

array([ 0.04079027,  0.08074931,  0.1203152 ,  0.15971658,  0.19880959,
        0.2378034 ,  0.27649712,  0.31503064,  0.35344849,  0.39162472,
        0.41593185,  0.42990261,  0.44353182,  0.45640762,  0.46917486,
        0.48188077,  0.49447547,  0.5070586 ,  0.51959347,  0.53208532,
        0.54451215,  0.55681476,  0.56751028,  0.57796498,  0.5880962 ,
        0.59765106,  0.60718088,  0.61664377,  0.62608944,  0.63549868,
        0.64485528,  0.65419884,  0.66352509,  0.67281533,  0.68201611,
        0.69115436,  0.70009723,  0.70854641,  0.7168691 ,  0.72491695,
        0.73283184,  0.74062176,  0.74826358,  0.75590125,  0.7635132 ,
        0.77109004,  0.77860634,  0.78608603,  0.79355206,  0.80099485,
        0.80841933,  0.81523581,  0.82165025,  0.82801829,  0.8332951 ,
        0.83829627,  0.84322965,  0.8481287 ,  0.85298991,  0.85772344,
        0.86243577,  0.86710539,  0.87177407,  0.87641204,  0.88101395,
        0.88558019,  0.88974527,  0.89368719,  0.89738322,  0.90

In [96]:
pcX=pest.components_

In [75]:
pcX.T.shape

(97230L, 70L)

In [84]:
XX.shape

(97230L, 201L)

In [127]:
keys

['lobby_type',
 'r1_hero',
 'r2_hero',
 'r3_hero',
 'r4_hero',
 'r5_hero',
 'd1_hero',
 'd2_hero',
 'd3_hero',
 'd4_hero',
 'd5_hero']

In [190]:
#params_grid={"n_estimators":[10,20,30]}

In [191]:
#gs = GridSearchCV(GradientBoostingClassifier(),param_grid=params_grid,cv=cv,scoring="roc_auc")

In [193]:
times = pd.DataFrame(columns=['n_estimators','time','score'],index=[10,20,30,60])

In [194]:
for n in [10,20,30,60]:
    params_grid={"n_estimators":[n]}
    gs = GridSearchCV(GradientBoostingClassifier(),param_grid=params_grid,cv=cv,scoring="roc_auc")
    start_time = datetime.datetime.now()
    #times['n_estimators:' + gs.param_grid.values()[0][0]] = 
    gs.fit(X_na,y)
    n_est=gs.param_grid.values()[0][0]
    et=(datetime.datetime.now() - start_time).seconds
    sc=gs.best_score_
    times.loc[n_est] = [n_est,et,sc]

#print 'Time elapsed:', datetime.datetime.now() - start_time
times

,n_estimators,time,score
10,10,63,0.66469
20,20,126,0.682274
30,30,184,0.689361
60,60,505,0.700428


In [19]:
op_times = pd.DataFrame(columns=['n_estimators','time','score'],index=[10,20,30,60])

In [35]:
for n in [10]:
    params_grid={"n_estimators":[n],'loss':['deviance'],'subsample':[0.2]}
    gs = GridSearchCV(GradientBoostingClassifier(),param_grid=params_grid,cv=cv,scoring="roc_auc")
    start_time = datetime.datetime.now()
    #times['n_estimators:' + gs.param_grid.values()[0][0]] = 
    gs.fit(X_na,y)
    n_est=gs.param_grid.values()[0][0]
    et=(datetime.datetime.now() - start_time).seconds
    sc=gs.best_score_
    
    op_times.loc[n_est] = [n_est,et,sc]

#print 'Time elapsed:', datetime.datetime.now() - start_time
op_times

,n_estimators,time,score
10,10,46,0.668749
20,NaN,NaN,NaN
30,30,122,0.684247
60,NaN,NaN,NaN


In [34]:
op_times

,n_estimators,time,score
10,10,44,0.670543
20,NaN,NaN,NaN
30,30,122,0.684247
60,NaN,NaN,NaN


In [23]:
op_times

,n_estimators,time,score
10,NaN,NaN,NaN
20,NaN,NaN,NaN
30,30,195,0.690125
60,NaN,NaN,NaN
